In [1]:
from bs4 import BeautifulSoup
import re
import urllib.request, urllib.error
import xlwt

In [2]:
findLink = re.compile(r'<a href="(.*?)">')  
findImgSrc = re.compile(r'<img.*src="(.*?)"', re.S)
findTitle = re.compile(r'<span class="title">(.*)</span>')
findRating = re.compile(r'<span class="rating_num" property="v:average">(.*)</span>')
findReview = re.compile(r'<span>(\d*)人评价</span>')
findInq = re.compile(r'<span class="inq">(.*)</span>')
findBd = re.compile(r'<p class="">(.*?)</p>', re.S)


In [3]:
def main():
    url = "https://movie.douban.com/top250?start="
    datalist = getData(url)
    savepath = "豆瓣电影Top250.xls"
    saveData(datalist,savepath)

In [4]:
def getData(baseurl):
    datalist = []
    for i in range(0,10):
        url = baseurl + str(i*25)
        html = askUrl(url)
        
    soup = BeautifulSoup(html,"html.parser")
    for item in soup.find_all('div',class_="item"):
        data = []
        item = str(item)
        link = re.findall(findLink,item)[0]
        data.append(link)
        
        imgSrc = re.findall(findImgSrc,item)[0]
        data.append(imgSrc)
        
        title = re.findall(findTitle,item)
        data.append(title)
        if len(title)==2:
            ctitle = title[0]
            data.append(ctitle)
            otitle = title[1].replace("/","")
            data.append(otitle)
        else:
            data.append(title[0])
            data.append(" ")
            
        rating = re.findall(findRating,item)[0]
        data.append(rating)
        
        review = re.findall(findReview,item)[0]
        data.append(review)
        
        inq = re.findall(findInq,item)
        data.append(inq)
        if len(inq)!=0:
            inq = inq[0].replace("。","")
            data.append(inq)
        else:
            data.append(" ")
            
        bd = re.findall(findBd,item)[0]
        bd = re.sub('<br(\s+)?/>(\s+)?', "", bd)
        bd = re.sub('/', "", bd)
        data.append(bd.strip())
        datalist.append(data)
    return datalist
    

In [5]:
def askUrl(url):
    head = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36"}
    request = urllib.request.Request(url,headers = head)
    html = ""
    
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e,"code"):
            print(e.code)
        if hasattr(e,"reason"):
            print(e.reason)
    return html

In [6]:
def saveData(datalist,savepath):
    print("Saving......")
    book = xlwt.Workbook(encoding="utf-8",style_compression=0)
    sheet = book.add_sheet('豆瓣电影 TOP250',cell_overwrite_ok=True)
    col = ("电影详情链接","图片链接","影片中文名","影片外国名","评分","评价数","概况","相关信息")
    for i in range(0,8):
        sheet.write(0,i,col[i])
    for i in range(0,250):
        data = datalist[i]
        for j in range(0,8):
            sheet.write(i+1,j,data[j])
    book.save(savepath)

In [7]:
if __name__ == "__main__":
    main()
    print("Finished Data Scraping!")

Saving......


IndexError: list index out of range